<a href="https://colab.research.google.com/github/FranciscoPretto/EA2/blob/entrega/Pretto_Francisco_ejercicio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introducción

# 2. Armado del ambiente

In [1]:
!pip install pycuda

# 3. Desarrollo




In [2]:
from datetime import datetime
tiempo_total = datetime.now()

import matplotlib.pyplot as plt
import math 
import numpy
from PIL import Image 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

key = "HACK"

#Encriptar

def encriptarMensaje():
    
    msg = """Geeks for Geeks"""

   #msg = """Geeks for Geeksdsadasdasdddddddd"""
    mensaje_cifrado = ""

    # track key indices 
    k_indx = 0
  
    msg_len = float(len(msg)) 
    msg_lst = list(msg) 
    key_lst = sorted(list(key)) 

    #Genero la lista con el orden de la clave
    orden_clave=[]
    for i in range(len(key)):
        orden_clave+=[key.index(key_lst[i])]

    print(orden_clave)
    # Calcular numeros de columnas
    col = len(key)

     # Calcular maximo numero de filas
    row = int(math.ceil(msg_len / col)) 

    # Agregar los caracteres de relleno en los lugares sobrantesd de la matriz

    fill_null = int((row * col) - msg_len) 
    msg_lst.extend('_' * fill_null)

    # Crear la matriz e insertar los datos

    matrix = [msg_lst[i: i + col]  
              for i in range(0, len(msg_lst), col)] 
    
    #Creo las variables para enviarlas

    matrix_cpu = numpy.asmatrix(matrix)
    vector_cpu = numpy.empty_like(msg_lst)
    clave_cpu = numpy.asarray(orden_clave)

    #Reservo el espacio en memoria de GPU
    matrix_gpu = cuda.mem_alloc(matrix_cpu.nbytes)
    vector_gpu = cuda.mem_alloc(vector_cpu.nbytes)
    clave_gpu = cuda.mem_alloc(clave_cpu.nbytes)

    #Copio la memoria a la GPU
    cuda.memcpy_htod(matrix_gpu,matrix_cpu)
    cuda.memcpy_htod(clave_gpu,clave_cpu)

    # CPU - Defino la función kernel que ejecutará en GPU.
    module = SourceModule("""
    __global__ void transposition_cipher( int filas, int columnas, int* matriz, int* clave, int* resultado)   
    {
        // Calculo las coordenadas del Thread en dos dimensiones.
        int idx = threadIdx.x + blockIdx.x*blockDim.x;
        int idy = threadIdx.y + blockIdx.y*blockDim.y;

          // Verifico que los Thread, esten dentro de las dimensiones de la matriz.

          if(idx < columnas && idy <  filas)
          {

              int curr_idy = clave[idy*2];
              int pos_res = (idy*filas)+idx;
              resultado[pos_res] = matriz[curr_idy+(idx*columnas)]; 
        }

    }
        
    """)

    # CPU - Genero la función kernel.
    kernel = module.get_function("transposition_cipher")


    dim_hilo_x = row
    dim_bloque_x = numpy.int( (col+dim_hilo_x-1) / dim_hilo_x )

    dim_hilo_y = col
    dim_bloque_y = numpy.int( (row+dim_hilo_y-1) / dim_hilo_y )

    print( "Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
    print( "Total de Thread: [", dim_hilo_x*dim_bloque_x, ",", dim_hilo_y*dim_bloque_y, " ]", " = ", dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

    kernel( numpy.int32(row), numpy.int32(col),matrix_gpu, clave_gpu,vector_gpu,block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1) )

    # GPU - Copio el resultado desde la memoria GPU.
    cuda.memcpy_dtoh( vector_cpu, vector_gpu )
    print(vector_cpu)
  

encriptarMensaje()


[1, 2, 0, 3]
Thread: [ 4 , 4  ], Bloque : [ 1 , 1 ]
Total de Thread: [ 4 , 4  ]  =  16
['e' ' ' ' ' 'k' 'e' 'f' 'G' 's' 'G' 's' 'r' 'e' 'k' 'o' 'e' '_']


# 4. Tabla de pasos

# 5. Conclusiones

#6. Bibliografía